In [1]:
import pandas as pd
import numpy as np
data = pd.read_csv('merged_hpd_census_pluto.csv')
print data.shape
data.head(5)

(609122, 215)


,Unnamed: 0,Median_income,UnitsRes,AssessTot,Avg_value_per_res_unit,BuildingAge,YearSinceLastAlter,Probs_in_complaint,ViolationIssued,Tot_A_violations,...,Month_3,Month_11,Month_12,Month_1,Month_2,Month_6,Month_9,Month_8,Month_5,Month_10
0,0,69514,2,7363,3681.5,75,75,1,0,0,...,0,0,1,0,0,0,0,0,0,0
1,1,69514,1,8060,8060.0,70,70,1,0,0,...,0,1,0,0,0,0,0,0,0,0
2,2,69514,1,8060,8060.0,70,70,5,0,0,...,0,1,0,0,0,0,0,0,0,0
3,3,69514,1,8060,8060.0,70,70,5,0,0,...,0,1,0,0,0,0,0,0,0,0
4,4,69514,1,8060,8060.0,70,70,5,0,0,...,0,1,0,0,0,0,0,0,0,0


In [2]:
#move target variable 'ViolationIssue' to the first column, rename it into 'Y', drop 'Unnamed: 0' column
data.insert(1, 'Y', data['ViolationIssued'])
data = data.drop(['ViolationIssued', 'Unnamed: 0'], axis = 1)
print data.shape
data.head(5)

(609122, 214)


,Y,Median_income,UnitsRes,AssessTot,Avg_value_per_res_unit,BuildingAge,YearSinceLastAlter,Probs_in_complaint,Tot_A_violations,Tot_B_violations,...,Month_3,Month_11,Month_12,Month_1,Month_2,Month_6,Month_9,Month_8,Month_5,Month_10
0,0,69514,2,7363,3681.5,75,75,1,0,0,...,0,0,1,0,0,0,0,0,0,0
1,0,69514,1,8060,8060.0,70,70,1,0,0,...,0,1,0,0,0,0,0,0,0,0
2,0,69514,1,8060,8060.0,70,70,5,0,0,...,0,1,0,0,0,0,0,0,0,0
3,0,69514,1,8060,8060.0,70,70,5,0,0,...,0,1,0,0,0,0,0,0,0,0
4,0,69514,1,8060,8060.0,70,70,5,0,0,...,0,1,0,0,0,0,0,0,0,0


In [3]:
num_variables = ['Median_income', 'UnitsRes', 'AssessTot', 'Avg_value_per_res_unit', 'BuildingAge', 
                'YearSinceLastAlter', 'Probs_in_complaint','Tot_A_violations','Tot_B_violations','Tot_C_violations']

for variable in num_variables:
    print data[variable].dtype, variable

#convert 'Median_income' into float; otherwise, sklearn throws a warning message
data['Median_income'] = data['Median_income'].astype(float)
print '\n', data['Median_income'].dtype

int64 Median_income
float64 UnitsRes
float64 AssessTot
float64 Avg_value_per_res_unit
float64 BuildingAge
float64 YearSinceLastAlter
float64 Probs_in_complaint
float64 Tot_A_violations
float64 Tot_B_violations
float64 Tot_C_violations

float64


In [4]:
from sklearn import cross_validation
from sklearn import preprocessing

split = cross_validation.ShuffleSplit(data.shape[0], n_iter=1, train_size = 0.5, test_size=.25, random_state = 1)

for train, test in split:
    train_index = train
    test_index = test
    validation_index = np.arange(data.shape[0])
    validation_index = set(validation_index) - set(train_index) - set(test_index)

In [5]:
data_train = data.ix[train_index,:]
data_test = data.ix[test_index,:]
data_validation = data.ix[validation_index,:]

print '\ndata_train shape', data_train.shape
print 'data_test shape', data_test.shape
print 'data_validation shape', data_validation.shape
print '\nshape_data', data.shape
print 'sum train + test + validation = ', data_train.shape[0] + data_test.shape[0] + data_validation.shape[0]


data_train shape (304561, 214)
data_test shape (152281, 214)
data_validation shape (152280, 214)

shape_data (609122, 214)
sum train + test + validation =  609122


In [6]:
for variable in num_variables:
    scaler = preprocessing.StandardScaler().fit(data_train[variable])
    data_train[variable] = scaler.transform(data_train[variable])
        
    #The scaler instance is used on test and validation data to transform it the same way it did on the training set
    data_test[variable] = scaler.transform(data_test[variable]) 
    data_validation[variable] = scaler.transform(data_validation[variable]) 

In [7]:
print '__train__: '
print data_train.head(5)
print '\n__test__: '
print data_test.head(5)
print '\n__validation__:'
print data_validation.head(5)

__train__: 
        Y  Median_income  UnitsRes  AssessTot  Avg_value_per_res_unit  \
183121  0      -0.459742 -0.106421  -0.121231               -0.235067   
311090  0       1.012679 -0.077475   0.153470                1.059009   
219723  0       0.187398 -0.178788  -0.141940               -0.298887   
133405  0      -1.056931 -0.113658  -0.087871                0.004162   
118799  0       0.194361 -0.065413  -0.084946               -0.120348   

        BuildingAge  YearSinceLastAlter  Probs_in_complaint  Tot_A_violations  \
183121     0.224118            0.711411           -0.508070         -0.362469   
311090    -3.515782           -1.927744           -0.508070         -0.302166   
219723     0.050169           -1.897409           -0.807011         -0.422772   
133405     0.658990           -0.866015           -0.807011          0.059649   
118799     0.267605            0.741746           -0.807011          0.119951   

        Tot_B_violations    ...     Month_3  Month_11  Month_1

In [8]:
data_train.to_csv('data/data_train.csv')
data_test.to_csv('data/data_test.csv')
data_validation.to_csv('data/data_validation.csv')

In [9]:
new = pd.read_csv('data/data_train.csv')
new.head(5)

,Unnamed: 0,Y,Median_income,UnitsRes,AssessTot,Avg_value_per_res_unit,BuildingAge,YearSinceLastAlter,Probs_in_complaint,Tot_A_violations,...,Month_3,Month_11,Month_12,Month_1,Month_2,Month_6,Month_9,Month_8,Month_5,Month_10
0,183121,0,-0.459742,-0.106421,-0.121231,-0.235067,0.224118,0.711411,-0.508070,-0.362469,...,0,0,0,0,1,0,0,0,0,0
1,311090,0,1.012679,-0.077475,0.153470,1.059009,-3.515782,-1.927744,-0.508070,-0.302166,...,0,0,0,0,0,1,0,0,0,0
2,219723,0,0.187398,-0.178788,-0.141940,-0.298887,0.050169,-1.897409,-0.807011,-0.422772,...,0,0,0,0,0,0,0,0,0,0
3,133405,0,-1.056931,-0.113658,-0.087871,0.004162,0.658990,-0.866015,-0.807011,0.059649,...,0,0,0,0,0,0,0,0,0,0
4,118799,0,0.194361,-0.065413,-0.084946,-0.120348,0.267605,0.741746,-0.807011,0.119951,...,0,0,0,0,0,0,0,0,0,0
